# Validação Cruzada

In [7]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score

In [5]:
#carregando o dataset

base = pd.read_csv('autos.csv', encoding='ISO-8859-1')

#removendo algumas colunas que não serão úteis

base = base.drop('dateCrawled', axis=1)
base = base.drop('dateCreated', axis=1)
base = base.drop('nrOfPictures', axis=1)
base = base.drop('postalCode', axis=1)
base = base.drop('lastSeen', axis=1)

#removendo outras colunas que possuem uma variabilidade muito grande

base['name'].value_counts()
base = base.drop('name', axis=1)

base['seller'].value_counts()
base = base.drop('seller', axis=1)

base['offerType'].value_counts()
base = base.drop('offerType', axis=1)

#removendo os valores inconsistentes

i1 = base.loc[base.price<=10]
base = base[base.price>10]

i2 = base.loc[base.price>350000]
base = base[base.price<350000]

#tratamento de dados ausentes

valores = {
    'vehicleType': 'limousine',
    'gearbox': 'manuell',
    'model': 'golf',
    'fuelType': 'benzine',
    'notRepairedDamage': 'nein'
}

base = base.fillna(value=valores)

previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

#tranformando os dados categóricos em dados numéricos

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_previsores = LabelEncoder()
previsores[:, 0] = labelencoder_previsores.fit_transform(previsores[:, 0])
previsores[:, 1] = labelencoder_previsores.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_previsores.fit_transform(previsores[:, 5])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_previsores.fit_transform(previsores[:, 9])
previsores[:, 10] = labelencoder_previsores.fit_transform(previsores[:, 10])

#transformando os valores utilizando o onehotencoder

encoder2 = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])],remainder='passthrough')
previsores = encoder2.fit_transform(previsores).toarray()

In [8]:
#função para criar a rede neural

def criarede():
    regressor = Sequential()
    regressor.add(Dense(units=158, activation='relu', input_dim=317))
    regressor.add(Dense(units=158, activation='relu'))
    regressor.add(Dense(units=1, activation='linear'))
    regressor.compile(loss='mean_absolute_error', optimizer='adam',
                    metrics=['mean_absolute_error'])
    
    return regressor

In [10]:
regressor = KerasRegressor(build_fn=criarede,
                           epochs=100,
                           batch_size=300)

resultados = cross_val_score(estimator=regressor,
                             X=previsores, y=preco_real,
                             cv=10, scoring='neg_mean_absolute_error')

media = resultados.mean()
desvio = resultados.std()

/home/lucasarenhardt/.local/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/lucasarenhardt/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4344.2285 - mean_absolute_error: 4344.2285
Epoch 2/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3508.5220 - mean_absolute_error: 3508.5220
Epoch 3/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3268.5852 - mean_absolute_error: 3268.5850
Epoch 4/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3044.2051 - mean_absolute_error: 3044.2051
Epoch 5/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2841.2356 - mean_absolute_error: 2841.2356
Epoch 6/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2816.2729 - mean_absolute_error: 2816.2729
Epoch 7/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2750.0164 - mean_absolute_error: 2750.0161
Epoch 8/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2709.8848 - mean_absolute_error: 2709.8848
Epoch 9/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2671.2898 - mean_absolute_error: 2671.2898
Epoch 10/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s

/home/lucasarenhardt/.local/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/lucasarenhardt/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4312.5254 - mean_absolute_error: 4312.5254
Epoch 2/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3467.6238 - mean_absolute_error: 3467.6238
Epoch 3/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3297.1785 - mean_absolute_error: 3297.1785
Epoch 4/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3033.8262 - mean_absolute_error: 3033.8262
Epoch 5/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2935.7476 - mean_absolute_error: 2935.7476
Epoch 6/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2844.3186 - mean_absolute_error: 2844.3186
Epoch 7/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2779.6255 - mean_absolute_error: 2779.6255
Epoch 8/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2736.7249 - mean_absolute_error: 2736.7249
Epoch 9/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2717.7183 - mean_absolute_error: 2717.7183
Epoch 10/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s

/home/lucasarenhardt/.local/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/lucasarenhardt/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Epoch 1/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4261.8066 - mean_absolute_error: 4261.8066
Epoch 2/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3454.1284 - mean_absolute_error: 3454.1284
Epoch 3/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3239.2986 - mean_absolute_error: 3239.2986
Epoch 4/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3037.1504 - mean_absolute_error: 3037.1504
Epoch 5/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2884.8516 - mean_absolute_error: 2884.8516
Epoch 6/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2798.7773 - mean_absolute_error: 2798.7773
Epoch 7/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2765.4260 - mean_absolute_error: 2765.4263
Epoch 8/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2703.1987 - mean_absolute_error: 2703.1987
Epoch 9/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2650.1401 - mean_absolute_error: 2650.1401
Epoch 10/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s

/home/lucasarenhardt/.local/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/lucasarenhardt/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4293.5303 - mean_absolute_error: 4293.5303
Epoch 2/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3462.7312 - mean_absolute_error: 3462.7314
Epoch 3/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3166.1777 - mean_absolute_error: 3166.1777
Epoch 4/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2967.7485 - mean_absolute_error: 2967.7485
Epoch 5/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2838.4443 - mean_absolute_error: 2838.4443
Epoch 6/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2778.7136 - mean_absolute_error: 2778.7136
Epoch 7/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2759.8826 - mean_absolute_error: 2759.8823
Epoch 8/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2722.4661 - mean_absolute_error: 2722.4661
Epoch 9/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2705.3455 - mean_absolute_error: 2705.3457
Epoch 10/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s

/home/lucasarenhardt/.local/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/lucasarenhardt/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4253.3867 - mean_absolute_error: 4253.3867
Epoch 2/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3431.8967 - mean_absolute_error: 3431.8967
Epoch 3/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3123.4780 - mean_absolute_error: 3123.4780
Epoch 4/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2929.9270 - mean_absolute_error: 2929.9270
Epoch 5/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2818.4795 - mean_absolute_error: 2818.4795
Epoch 6/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2803.0984 - mean_absolute_error: 2803.0984
Epoch 7/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2747.3621 - mean_absolute_error: 2747.3621
Epoch 8/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2695.4182 - mean_absolute_error: 2695.4182
Epoch 9/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2655.0862 - mean_absolute_error: 2655.0862
Epoch 10/100
1078/1078 ━━━━━━━━━━━━━━━━━━━━ 3s

In [11]:
print(media)
print(desvio)

-2248.447364606603
38.31267932721663
